In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf


conf = SparkConf().set('spark.ui.port', '4050')
sc = SparkContext(conf=conf)
spark = SparkSession.builder.master('local[*]').getOrCreate()

## Loading CSV files into **RDD**

In [ ]:
IMDB_movies_df=spark.read.option("header",True) \
     .csv("/content/contentdrive/MyDrive/data_for_assignment/IMDb movies.csv")

In [ ]:
IMDB_movies_df.columns

['imdb_title_id',
 'title',
 'original_title',
 'year',
 'date_published',
 'genre',
 'duration',
 'country',
 'language',
 'director',
 'writer',
 'production_company',
 'actors',
 'description',
 'avg_vote',
 'votes',
 'budget',
 'usa_gross_income',
 'worlwide_gross_income',
 'metascore',
 'reviews_from_users',
 'reviews_from_critics']

In [ ]:
from pyspark.sql.functions import regexp_replace
IMDB_movies_df_cleansed=IMDB_movies_df.select(  regexp_replace('imdb_title_id',',','-').alias('imdb_title_id'),
 regexp_replace('title',',','-').alias( 'title'),
 regexp_replace('original_title',',','-').alias( 'original_title'),
 regexp_replace('year',',','-').alias( 'year'),
 regexp_replace('date_published',',','-').alias( 'date_published'),
 regexp_replace('genre',',','-').alias( 'genre'),
 regexp_replace('duration',',','-').alias( 'duration'),
 regexp_replace('country',',','-').alias( 'country'),
 regexp_replace('language',',','-').alias( 'language'),
 regexp_replace('director',',','-').alias( 'director'),
 regexp_replace('writer',',','-').alias( 'writer'),
 regexp_replace('production_company',',','-').alias( 'production_company'),
 regexp_replace('actors',',','-').alias( 'actors'),
 regexp_replace('description',',','-').alias( 'description'),
 regexp_replace('avg_vote',',','-').alias( 'avg_vote'),
 regexp_replace('votes',',','-').alias( 'votes'),
 regexp_replace('budget',',','-').alias( 'budget'),
 regexp_replace('usa_gross_income',',','-').alias( 'usa_gross_income'),
 regexp_replace('worlwide_gross_income',',','-').alias( 'worlwide_gross_income'),
 regexp_replace('metascore',',','-').alias( 'metascore'),
 regexp_replace('reviews_from_users',',','-').alias( 'reviews_from_users'),
 regexp_replace('reviews_from_critics',',','-').alias('reviews_from_critics'))

In [ ]:
IMDB_movies_df_cleansed.show()

+-------------+--------------------+--------------------+----+--------------+--------------------+--------+----------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+-----+----------+----------------+---------------------+---------+------------------+--------------------+
|imdb_title_id|               title|      original_title|year|date_published|               genre|duration|         country|       language|            director|              writer|  production_company|              actors|         description|avg_vote|votes|    budget|usa_gross_income|worlwide_gross_income|metascore|reviews_from_users|reviews_from_critics|
+-------------+--------------------+--------------------+----+--------------+--------------------+--------+----------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+-----+----------+-------

In [ ]:
#creating rdds from df
IMDB_movies=IMDB_movies_df_cleansed.rdd

In [ ]:
type(IMDB_movies)

pyspark.rdd.RDD

In [ ]:
#caching in memory using cache method for future use
IMDB_movies.cache()

MapPartitionsRDD[54] at javaToPython at NativeMethodAccessorImpl.java:0

In [ ]:
IMDB_ratings=sc.textFile('/content/contentdrive/MyDrive/data_for_assignment/IMDb ratings.csv')
type(IMDB_ratings)

pyspark.rdd.RDD

In [ ]:
#caching in memory using cache method for future use
IMDB_ratings.cache()

/content/contentdrive/MyDrive/data_for_assignment/IMDb ratings.csv MapPartitionsRDD[5] at textFile at NativeMethodAccessorImpl.java:0

# 1)	Brief the column information provided in each datasets

In [ ]:
for i in IMDB_movies.take(1):
  for j in i.asDict().keys():
    print(j)
  break

imdb_title_id
title
original_title
year
date_published
genre
duration
country
language
director
writer
production_company
actors
description
avg_vote
votes
budget
usa_gross_income
worlwide_gross_income
metascore
reviews_from_users
reviews_from_critics


In [ ]:
ratings_header=IMDB_ratings.first()
ratings_header_list=ratings_header.split(',')
for i  in ratings_header_list:
  print(i)

imdb_title_id
weighted_average_vote
total_votes
mean_vote
median_vote
votes_10
votes_9
votes_8
votes_7
votes_6
votes_5
votes_4
votes_3
votes_2
votes_1
allgenders_0age_avg_vote
allgenders_0age_votes
allgenders_18age_avg_vote
allgenders_18age_votes
allgenders_30age_avg_vote
allgenders_30age_votes
allgenders_45age_avg_vote
allgenders_45age_votes
males_allages_avg_vote
males_allages_votes
males_0age_avg_vote
males_0age_votes
males_18age_avg_vote
males_18age_votes
males_30age_avg_vote
males_30age_votes
males_45age_avg_vote
males_45age_votes
females_allages_avg_vote
females_allages_votes
females_0age_avg_vote
females_0age_votes
females_18age_avg_vote
females_18age_votes
females_30age_avg_vote
females_30age_votes
females_45age_avg_vote
females_45age_votes
top1000_voters_rating
top1000_voters_votes
us_voters_rating
us_voters_votes
non_us_voters_rating
non_us_voters_votes


# 2)	Give the distinct languages in which the movies were listed in IMDB

In [ ]:
IMDB_movies.map(lambda rec:rec.year).distinct().collect()

['1894',
 '1906',
 '1911',
 '1912',
 '1919',
 '1913',
 '1914',
 '1915',
 '1916',
 '1917',
 '1918',
 '1920',
 '1921',
 '1924',
 '1922',
 '1923',
 '1925',
 '1926',
 '1935',
 '1927',
 '1928',
 '1983',
 '1929',
 '1930',
 '1932',
 '1931',
 '1937',
 '1938',
 '1933',
 '1934',
 '1936',
 '1940',
 '1939',
 '1942',
 '1943',
 '1941',
 '1948',
 '1944',
 '2001',
 '1946',
 '1945',
 '1947',
 '1973',
 '1949',
 '1950',
 '1952',
 '1951',
 '1962',
 '1953',
 '1954',
 '1955',
 '1961',
 '1956',
 '1958',
 '1957',
 '1959',
 '1960',
 '1963',
 '1965',
 '1971',
 '1964',
 '1966',
 '1968',
 '1967',
 '1969',
 '1976',
 '1970',
 '1979',
 '1972',
 '1981',
 '1978',
 '2000',
 '1989',
 '1975',
 '1974',
 '1986',
 '1990',
 '2018',
 '1977',
 '1982',
 '1980',
 '1993',
 '1984',
 '1985',
 '1988',
 '1987',
 '2005',
 '1991',
 '2002',
 '1994',
 '1992',
 '1995',
 '2017',
 '1997',
 '1996',
 '2006',
 '1999',
 '1998',
 '2007',
 '2008',
 '2003',
 '2004',
 '2010',
 '2009',
 '2011',
 '2013',
 '2012',
 '2016',
 '2015',
 '2014',
 '2019',
 

# 3)	Give the number of movies which were not released in English

In [ ]:
movies_lang=IMDB_movies.map(lambda rec: str(rec.language).upper())

In [ ]:
movies_lang.take(3)

['NONE', 'NONE', 'NONE']

In [ ]:
sc.setCheckpointDir('/content/contentdrive/MyDrive/data_for_assignment')
movies_lang.checkpoint()

In [ ]:
movies_lang.filter(lambda lang: 'ENGLISH' not in lang).count()

38409

# 4)	Give the unique no of languages in which the films are being released

In [ ]:
lang_flat=movies_lang.flatMap(lambda lang:lang.split('-'))
lang_flat.take(3)

['NONE', 'NONE', 'NONE']

In [ ]:
lang_flat.map(lambda rec:rec.strip()).distinct().take(10)

['NONE',
 'ENGLISH',
 'ITALIAN',
 'GERMAN',
 'DANISH',
 'FRENCH',
 'HUNGARIAN',
 'RUSSIAN',
 'SPANISH',
 'DUTCH']

In [ ]:
lang_flat.map(lambda rec:rec.strip()).distinct().count()

269

# 5)	Name the top 5 lengthiest and 5 shortest movies along with duration from the list registered in IMDB

In [ ]:
duration_rdd_desc=IMDB_movies.sortBy(lambda rec: int(rec.duration)*-1)
duration_rdd_asc=IMDB_movies.sortBy(lambda rec: int(rec.duration))

In [ ]:
duration_rdd_desc.map(lambda rec: (rec.title,rec.duration)).take(5)

[('La flor', '808'),
 ('Out 1- noli me tangere', '729'),
 ('Khleb - imya sushchestvitelnoe', '580'),
 ('Orbius', '570'),
 ('Ebolusyon ng isang pamilyang Pilipino', '540')]

In [ ]:
duration_rdd_asc.map(lambda rec: (rec.title,rec.duration)).take(5)

[('Dragon Ball Z: La vendetta divina', '41'),
 ('Enigma', '42'),
 ('Niagara Falls', '43'),
 ('My Little Pony: Equestria Girls - Holidays Unwrapped', '44'),
 ('Miss Jerry', '45')]

In [ ]:
'4.5'.isdigit()

False

# 6) Get top 5 films for which the avg_vote is equal to average of all available ratings of all movies

In [ ]:
vote_rdd=IMDB_movies.map(lambda rec: rec.avg_vote).\
    filter(lambda rec:rec.replace('.','').isdigit()) # replace is used sing the values are in floats
sum_avg_vote=vote_rdd.reduce(lambda x,y: float(x)+float(y))
avg_votes=round(sum_avg_vote/vote_rdd.count(),1)
print(avg_votes)
avg_rated_movies=IMDB_movies.map(lambda rec: (rec.title,rec.avg_vote)).\
    filter(lambda rec: str(avg_votes) in rec[1])
avg_rated_movies.take(5)

5.9


[('Miss Jerry', '5.9'),
 ('Maciste', '5.9'),
 ('Young Romance', '5.9'),
 ('His Picture in the Papers', '5.9'),
 ("'49-'17", '5.9')]

# 7)	Need the list of all movies which contains the average vote calculated above and  write the result into a textfile

In [ ]:
avg_rated_movies.saveAsTextFile('/content/contentdrive/MyDrive/avg_rated_movies.txt')

# 8)	Take a sample partition of approximately 1000 rows and get average no of votes casted for that sample

In [ ]:
IMDB_movies.count()

85855

In [ ]:
IMDB_movies_part=IMDB_movies.repartition(85)

In [ ]:
IMDB_movies_part.getNumPartitions()

85

In [ ]:
for i in IMDB_movies_part.glom().take(1):
  cnt=0
  for j in i:
    cnt=cnt+int(j.votes) if j.votes.isdigit() else 0
print(round(cnt/len(i)))

34


# 9)	Calculate all the average of men_avg_vote for every distinct  year released movies in the list

In [ ]:
IMDB_ratings_woh=IMDB_ratings.filter(lambda x: x!= ratings_header).\
  map(lambda rec:(rec.split(',')[0],rec.split(',')[23]))#23 is the index fora all_men_avg_vote

In [ ]:
IMDB_ratings_woh.take(3)

[('tt0000009', '6.2'), ('tt0000574', '6.1'), ('tt0001892', '5.9')]

In [ ]:
movies_year=IMDB_movies.map(lambda rec: (rec.imdb_title_id,rec.year))

In [ ]:
movies_year.take(2)

[('tt0000009', '1894'), ('tt0000574', '1906')]

In [ ]:
movies_year.join(IMDB_ratings_woh).take(2)

[('tt0000009', ('1894', '6.2')), ('tt0000574', ('1906', '6.1'))]

In [ ]:
req_rdd=movies_year.join(IMDB_ratings_woh).map(lambda rec: rec[1]).\
    filter(lambda rec:rec[1].replace('.','').isdigit()).\
    map(lambda rec : (rec[0],float(rec[1])))
req_rdd.count()

85854

In [ ]:

sample = (0,0)
inter = req_rdd.aggregateByKey(sample, lambda a,b: (a[0] + b,    a[1] + 1),
                                       lambda a,b: (a[0] + b[0], a[1] + b[1]))
inter.take(2)


[('1911', (30.9, 5)), ('1913', (84.20000000000002, 13))]

In [ ]:
inter.mapValues(lambda v: round(v[0]/v[1],1)).collect()

[('1911', 6.2),
 ('1913', 6.5),
 ('1918', 6.2),
 ('1920', 6.2),
 ('1921', 6.6),
 ('1925', 6.6),
 ('1927', 6.6),
 ('1983', 6.0),
 ('1928', 6.8),
 ('1937', 6.4),
 ('1934', 6.2),
 ('1936', 6.3),
 ('1938', 6.3),
 ('1940', 6.4),
 ('1939', 6.5),
 ('1943', 6.4),
 ('1941', 6.4),
 ('1947', 6.5),
 ('1948', 6.5),
 ('1949', 6.5),
 ('1951', 6.4),
 ('1962', 6.4),
 ('1954', 6.3),
 ('1956', 6.3),
 ('1957', 6.3),
 ('1964', 6.3),
 ('1970', 6.1),
 ('1979', 6.0),
 ('1969', 6.1),
 ('1981', 6.1),
 ('1978', 6.1),
 ('1977', 6.1),
 ('1982', 6.0),
 ('1980', 6.0),
 ('1990', 5.9),
 ('1991', 5.9),
 ('2002', 5.8),
 ('1994', 5.8),
 ('1995', 5.8),
 ('1993', 5.9),
 ('2003', 5.7),
 ('2017', 5.5),
 ('2010', 5.6),
 ('2004', 5.8),
 ('2011', 5.5),
 ('2007', 5.7),
 ('2013', 5.5),
 ('2012', 5.5),
 ('2015', 5.5),
 ('2014', 5.6),
 ('2019', 5.6),
 ('2016', 5.5),
 ('2020', 5.3),
 ('1894', 6.2),
 ('1906', 6.1),
 ('1912', 6.0),
 ('1914', 6.1),
 ('1915', 6.3),
 ('1917', 6.3),
 ('1916', 6.4),
 ('1919', 6.4),
 ('1924', 6.7),
 ('1922'